In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [227]:
# 기획전략, 1page 로 임의로 설정
URL = 'https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart=10038&isSaved=1&Page=2'
header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
response = requests.get(URL, headers = header)

In [228]:
response

<Response [200]>

In [233]:
soup = BeautifulSoup(response.text, 'html.parser')
tags=soup.select('.selfLists li div p') # 목록사이트에 있는 공고개수
tags

[]

In [234]:
len(tags)

0

## 최종 코드 & Columns 설정
### 지원분야(section), 기업(company), 지원시기(date), 신입/경력(work_type), 직무(job), 문항(question), 답변(answer)

In [254]:
schPart_dict = { "기획·전략" : 10026, "법무·사무·총무" : 10027, "인사·HR" :10028,
 "회계·세무" :10029, "마케팅·광고·MD" :10030,  "개발·데이터" : 10031,
 "디자인" :10032, "물류·무역" :10033, "운전·운송·배송" :10034,
 "영업" : 10035, "고객상담·TM" :10036, "금융·보험" :10037, "식·음료" :10038,
 "고객서비스·리테일" : 10039, "엔지니어링·설계" :10040, "제조·생산" :10041, "교육" :10042,
 "건축·시설" :10043, "의료·바이오" :10044, "미디어·문화·스포츠" :10045, "공공·복지" :10046}

In [255]:
df= pd.DataFrame(columns= ['section','company','date','work_type','job','question','answer'])
for kor_cate, num_cate in schPart_dict.items(): #ex) 기획·전략 10026
    for page in range(1,100): # 영업이 최대페이수 보유. 81페이지. 
        BASE_URL = 'https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart={}&isSaved=1&Page={}'
        URL= BASE_URL.format(num_cate,page)
        response = requests.get(URL, headers = header)
        soup = BeautifulSoup(response.text, 'html.parser')
        tags = soup.select('.selfLists li div p') # 목록사이트에 있는 공고개수
        
        if len(tags)==0 : # 목록의 개수가 0이라는 것은 마지막 페이지를 넘어갔다는 뜻!
                break # 다음 section으로 이동! 
                
        for tag in tags : 
            company = tag.select_one('a .titTx').text.strip() # 기업명
            date = tag.select_one('.linkArray .career').text.strip() # 지원시기
            work_type = tag.select_one('.linkArray .field').text.strip() # 신입/경력    

            # 자소서 세부 url로 들어가기
            url = 'https://www.jobkorea.co.kr/'+ tag.find('a').get('href') # 세부 url
            response= requests.get(url, headers= header)
            soup= BeautifulSoup(response.text, 'html.parser')

            # job에 대한 부분이 없는 경우 존재해서 try, except
            try : 
                job = soup.select_one('.viewTitWrap .hd em').text.strip().split()[-1] # 직무
            except:
                job = '' 
            questions= soup.select('.qnaLists dt .tx') # 문항_리스트
            answers = soup.select('.qnaLists dd .tx') # 답변_리스트
            counts = soup.select('.txSpllChk') # 글자수, bytes수 정보  

            # /r 없애기
            for j in range(len(answers)):
                # 아쉬운점 mark 없애기
                question= questions[j].text #문항
                answer = answers[j].text.replace('\r','').strip()
                answer = answer.replace(counts[j].text,'').strip() #답변

                # 좋은점, 아쉬운점 없애기
                try:
                    advice_marks= soup.select('.qnaLists dd .tx .sup') #아쉬운점 
                    for am in advice_marks :
                        answer= answer.replace(am.text, '') 
                except : 
                    pass

                df.loc[len(df)]= [kor_cate, company, date, work_type, job, question, answer] # df에 정보 넣기! 
            
        if len(tags)<20 : # 목록의 개수가 20보다 작다는것은 해당 section의 마지막 페이지라는 뜻
                break # 다음 section으로 이동! 




In [256]:
df

,section,company,date,work_type,job,question,answer
0,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,오스템임플란트에 입사 지원한 동기/이유를 설명해 주세요.,"""현실이 주는 안락함에 도취하지 않고 도약을 멈추지 않는 오스템임플란트"" 임플란트 ..."
1,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,과거 경험 中 본인의 실패 사례를 소개해 주세요.,"""감성과 이성을 아울러 성공을 향해 함께 나아가다."" OOOOO 차장으로서 OOOO..."
2,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,본인의 열정 발휘 사례를 소개해 주세요.,"""소통이 중심이 된 기획력과 문제해결력으로 비효율성을 해결하다."" 대대 인사행정병으..."
3,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,입사 후 지원 분야에 본인이 기여할 수 있는 방안에 대해 설명해 주세요.,"""다채로운 관련 경험으로 습득한 학술행사 노하우"" 학교, 군대, 지자체를 아우르며 ..."
4,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,입사 지원 분야에 대한 본인의 직무 수행 경험을 설명해 주세요.,"""경험을 바탕으로 성장하는 제너럴리스트"" 주도적으로 다양한 분야에서 행사 기획과 운..."
...,...,...,...,...,...,...,...
29484,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"봉사활동이나 프로젝트 모임, 동아리, 현장 실습이나 인턴 기업체 등과 같이 낯선 단...","[꾸준한 반복의 힘] G1 강원민방에서 단종뎐, 민심에 살아있는 임금이라는 프로그램..."
29485,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,자신이 속한 팀이나 집단 전체의 이익을 위하여 주도적으로 행동하고 노력하는 사람임을...,[팀원을 하나로 만드는 원칙] 대학생 봉사단의 영상팀장으로 활동하며 CSR 필름 페...
29486,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"[의사소통능력] 자신이 속했던 조직(학교, 회사, 동아리 등) 안에서 자신과 의견이...","복지관에서 실습을 할 때, 어르신이 가진 편견을 깨고 수입조사를 무사히 끝낸 경험이..."
29487,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"[조직이해능력] 봉사활동이나 프로젝트 모임, 동아리, 현장 실습이나 인턴 기업체 등...","노인복지관에서 봉사활동을 할 때, 프로그램 접수업무를 맡아서 진행한 적이 있습니다...."


In [260]:
# csv로 저장하는 코드
df.to_csv('jobkorea.csv',index=False)

In [261]:
#csv df로 불러오기
pd.read_csv('jobkorea.csv')

,section,company,date,work_type,job,question,answer
0,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,오스템임플란트에 입사 지원한 동기/이유를 설명해 주세요.,"""현실이 주는 안락함에 도취하지 않고 도약을 멈추지 않는 오스템임플란트"" 임플란트 ..."
1,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,과거 경험 中 본인의 실패 사례를 소개해 주세요.,"""감성과 이성을 아울러 성공을 향해 함께 나아가다."" OOOOO 차장으로서 OOOO..."
2,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,본인의 열정 발휘 사례를 소개해 주세요.,"""소통이 중심이 된 기획력과 문제해결력으로 비효율성을 해결하다."" 대대 인사행정병으..."
3,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,입사 후 지원 분야에 본인이 기여할 수 있는 방안에 대해 설명해 주세요.,"""다채로운 관련 경험으로 습득한 학술행사 노하우"" 학교, 군대, 지자체를 아우르며 ..."
4,기획·전략,오스템임플란트(주),2022년 하반기,신입,경영·비즈니스기획,입사 지원 분야에 대한 본인의 직무 수행 경험을 설명해 주세요.,"""경험을 바탕으로 성장하는 제너럴리스트"" 주도적으로 다양한 분야에서 행사 기획과 운..."
...,...,...,...,...,...,...,...
29484,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"봉사활동이나 프로젝트 모임, 동아리, 현장 실습이나 인턴 기업체 등과 같이 낯선 단...","[꾸준한 반복의 힘] G1 강원민방에서 단종뎐, 민심에 살아있는 임금이라는 프로그램..."
29485,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,자신이 속한 팀이나 집단 전체의 이익을 위하여 주도적으로 행동하고 노력하는 사람임을...,[팀원을 하나로 만드는 원칙] 대학생 봉사단의 영상팀장으로 활동하며 CSR 필름 페...
29486,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"[의사소통능력] 자신이 속했던 조직(학교, 회사, 동아리 등) 안에서 자신과 의견이...","복지관에서 실습을 할 때, 어르신이 가진 편견을 깨고 수입조사를 무사히 끝낸 경험이..."
29487,공공·복지,국민건강보험공단,2016년 상반기,신입,사회복지사,"[조직이해능력] 봉사활동이나 프로젝트 모임, 동아리, 현장 실습이나 인턴 기업체 등...","노인복지관에서 봉사활동을 할 때, 프로그램 접수업무를 맡아서 진행한 적이 있습니다...."
